In [1]:
# inspired by
# https://github.com/fjg00/Facebook-Group-Post-Scraper/blob/main/Facebook%20Group%20Parser.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup

import argparse
import time
import json

import os
from selenium.webdriver.chrome.options import Options

In [2]:
# USER DEFINED INPUTS
import Login
CHROME_DRIVER_PATH = './chromedriver' # must download this yourself

# EMAIL, PASSWORD from Login file 
EMAIL = Login.EMAIL
PASSWORD = Login.PASSWORD

# Group number
GROUP_ID = 1379345962387168 #USER INPUT
GROUP_URL = 'https://www.facebook.com/groups/'+str(GROUP_ID)

SEARCH_PROMPT = "" # something to be searched for in the group
SEARCH_PROMPT = SEARCH_PROMPT.replace(" ","%20")

# Copy the group description and put it here to be able to filter it out
GROUP_DESCRIPTION = """-Kamer in Eindhoven-
Kamer in Eindhoven is een platform waar vraag en aanbod (kosteloos) worden samengebracht. Ben je opzoek naar een kamer, huisgenoot of bied je een kamer aan, word dan nu lid van deze facebookgroep.
Je scrollt door de nieuwste kamers en huisgenoten om vervolgens direct te reageren naar de aanbieder. Bij ons geen inschrijfkosten, gewoon studenten onder elkaar.
Kamer in Eindhoven is onderdeel van de Facebook community Zoekt Kamer in… Wij zijn ook actief in Nijmegen, Amsterdam, Delft, Rotterdam, Groningen, Maastricht, Breda, Utrecht, Leiden, Den Haag, Haarlem en Amersfoort.
Sinds 2020 hebben wij een samenwerking met de gratis app MyHospi. Hiermee willen wij het proces voor jullie nog makkelijker maken.
HOE WERKT MYHOSPI VOOR HUIZEN?
1. Plaats de kamer op myHospi via de app.
2. Deel de link naar het huis in je FB bericht en geef aan dat mensen via myHospi (de link) moeten reageren. Met myHospi krijg je direct een compleet beeld van de mensen die reageren op jouw kamer. Geen onpersoonlijke mails maar complete profielen.
3. Swipe en Like!
4. Nodig kandidaten uit en deel de link naar je videocall.""" # User input

In [3]:
def addvalues(dictionary, key, L):
    """Append multiple values to a key in the given dictionary"""
    if key not in dictionary:
        dictionary[key] = list()
    dictionary[key].extend(L)
    return dictionary


In [4]:
# Webdriver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-notifications')

driver = webdriver.Chrome(CHROME_DRIVER_PATH, options=chrome_options) #USER INPUT
driver.get('https://www.facebook.com/')

WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='_42ft _4jy0 _9xo6 _4jy3 _4jy1 selected _51sy']"))).click()
email = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name = 'email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name = 'pass']")))


email.clear()
password.clear()

email.send_keys(EMAIL) #USER INPUT
password.send_keys(PASSWORD) #USER INPUT

/tmp/ipykernel_16436/2054855826.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(CHROME_DRIVER_PATH, options=chrome_options) #USER INPUT


In [5]:
# Open window, select group
login = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[type = 'submit']"))).click()

#maximize window
driver.maximize_window()

n_scrolls = 2

# Gets FB group with search parameters
driver.get(GROUP_URL+SEARCH_PROMPT)

In [6]:
# Selection criteria
text = ""
comments = ""
z = 1

In [7]:
# Scroll through the page and get all links
all_urls = []
for i in range(1,n_scrolls):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(5)
    
    
    a_tags = driver.find_elements(By.TAG_NAME, 'a')
    all_urls.extend([a.get_attribute('href') for a in a_tags])
    

In [8]:
len(all_urls)

163

In [9]:
# Add link to anchor list if it's a group post
comments = [a for a in all_urls if (str(a).startswith(GROUP_URL+'/posts/'))]
posts = set(comment.partition('?comment')[0] for comment in comments)
print ('post count', len(posts))
print ('comment count', len(comments))
print ('all_urls count', len(all_urls))
# Current result with n = 20

post count 4
comment count 8
all_urls count 163


In [10]:
# Grab post and comment data
# TODO make nice df out of it
# TODO grab info about posters and commenters
# TODO grab time data (and time difference between post and comment)
WrapperDict = list() 
    
for a in posts:
        driver.get(a)
        time.sleep(1)
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        actualPosts =  soup.find('div' , class_ = "xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs x126k92a") #posts
        isTranslated = len(soup.find_all('span', class_ = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1pg5gke xvq8zen xo1l8bm x1qq9wsj x1yc453h")) > 0
        commentCount = soup.find('span', class_ = "x193iq5w xeuugli x13faqbe x1vvkbs x10flsy6 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x41vudc x6prxxf xvq8zen xo1l8bm xi81zsa")

        # TODO check that ALL comments are actually found
        # Get comments
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[@class='x78zum5 x1w0mnb xeuugli']"))).click()
        # TODO handle this (see print statement below) for text AND for emoji usage
        actualComments = soup.find_all('div', class_ = "xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs")
        
        # TODO create DF/dictionary/smth with all this info

        print ('actualPosts text', actualPosts.text)
        print ('isTranslated', isTranslated)
        print ('commentCount', commentCount)
        # print ('actualComments', actualComments[0])
        print ('actualComments count', len(actualComments))
        for i in range(0, len(actualComments)):
            print ('ACTUAL actual comment', actualComments[i].div.string)

        w = 1
        
        BranchDict = dict()
        BranchDict["tag"] = 1
        BranchDict["patterns"] = 1
        BranchDict["responses"] = list()
        L = list()
        
        for post in actualPosts:
            s = post.get_text()
            # if (s == "INSERT GROUP DESCRIPTION HERE"): #USER INPUT
            if (s == GROUP_DESCRIPTION): #USER INPUT
                time.sleep(1)
            elif len(s.split()) < 4:
                time.sleep(1)
            else:
                if w == 1: #post
                    text= text + post.get_text() 
                    BranchDict["tag"] = SEARCH_PROMPT + str(z) 
                    BranchDict["patterns"] = text
                    w = w + 1
                elif w > 1 : #comments
                    comments = post.get_text()
                    L.append(comments)
                    comments = ""
        z = z + 1
        addvalues(BranchDict, "responses", L)
        WrapperDict.append(BranchDict)
        text = ""

actualPosts text [Rental November-February in Strijp, viewing evening on Friday 4 November]Ha future house mate!We are from California and we are looking for a housemate to join us from November to February (with a view to permanent)The room in our cozy house at Strijp-S has a surface area of 15m2 and its own wash basin. The room is located on the second floor of the house. In addition to the beautiful room, the price also includes 4 roommates: Vera, Matthijs, Tijn and Eefje. We are a relaxed house in which everyone enjoys doing what he/she wants. Well you can join us almost every evening to eat together on the roof terrace and the doors are almost always open for a chat, to watch a series or a game.Station Strijp-S, the bus stop and the supermarket are within walking distance and you are within a few steps to the nice Strijp-s. In addition, you are within 15 minutes by bike at the TU, Fontys or in the city. The furniture in the room just stays. This entire package is yours for €375/mo

NameError: name 'x' is not defined

In [ ]:
# # Save it, brother
# x = '/.' + x + '.json'

# with open(x,'w', encoding='utf-8') as file:
#     file.write(json.dumps(WrapperDict, indent = 4, ensure_ascii=False).encode('utf-8').decode())
